In [ ]:
import json
from monitoring_utils import get_username

In [ ]:
USERNAME = get_username()
repository_name = f"utec-mlops/evidentlyai/server/{USERNAME}"
repository_name

'utec-mlops/evidentlyai/server'

In [3]:
!aws ecr create-repository --repository-name $repository_name


An error occurred (RepositoryAlreadyExistsException) when calling the CreateRepository operation: The repository with name 'utec-mlops/evidentlyai/server' already exists in the registry with id '654654589924'


In [4]:
!docker build --network sagemaker --no-cache -t $repository_name .

DEPRECATED: The legacy builder is deprecated and will be removed in a future release.
            BuildKit is currently disabled; enable it by removing the DOCKER_BUILDKIT=0
            environment-variable.

Sending build context to Docker daemon  30.58MB
Step 1/8 : FROM python:3.10-slim
3.10-slim: Pulling from library/python

7da3f26b: Pulling fs layer 
b4f4fa5c: Pulling fs layer 
56a18493: Pulling fs layer 
Digest: sha256:034724ef64585eeb0e82385e9aabcbeabfe5f7cae2c2dcedb1da95114372b6d7
Status: Downloaded newer image for python:3.10-slim
 ---> cce3651534b6
Step 2/8 : ENV PYTHONDONTWRITEBYTECODE 1
 ---> Running in 236bd8da813f
 ---> Removed intermediate container 236bd8da813f
 ---> f2df952b109a
Step 3/8 : ENV PYTHONUNBUFFERED 1
 ---> Running in d07f43908f5b
 ---> Removed intermediate container d07f43908f5b
 ---> d6727d583fb4
Step 4/8 : WORKDIR /app
 ---> Running in 73c03889283b
 ---> Removed intermediate container 73c03889283b
 ---> e2d30c395a1e
Step 5/8 : RUN apt-get update && apt-ge

In [5]:
!docker images

REPOSITORY                      TAG       IMAGE ID       CREATED        SIZE
utec-mlops/evidentlyai/server   latest    57e407ad94cd   1 second ago   1.2GB


In [6]:
ecr_url = "654654589924.dkr.ecr.us-east-1.amazonaws.com"

In [7]:
!aws ecr get-login-password --region us-east-1 | docker login --username AWS --password-stdin $ecr_url

WARNING! Your password will be stored unencrypted in /home/sagemaker-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credential-stores

Login Succeeded


In [8]:
tag_name = f"{repository_name}:latest"
full_tag_name = f"{ecr_url}/{tag_name}"
print(tag_name)
print(full_tag_name)

utec-mlops/evidentlyai/server:latest
654654589924.dkr.ecr.us-east-1.amazonaws.com/utec-mlops/evidentlyai/server:latest


In [9]:
!docker tag $tag_name $full_tag_name

In [10]:
!docker images

REPOSITORY                                                                   TAG       IMAGE ID       CREATED         SIZE
654654589924.dkr.ecr.us-east-1.amazonaws.com/utec-mlops/evidentlyai/server   latest    57e407ad94cd   4 seconds ago   1.2GB
utec-mlops/evidentlyai/server                                                latest    57e407ad94cd   4 seconds ago   1.2GB


In [11]:
!docker push $full_tag_name

The push refers to repository [654654589924.dkr.ecr.us-east-1.amazonaws.com/utec-mlops/evidentlyai/server]

93f39be0: Preparing 
b0b56a25: Preparing 
36139db3: Preparing 
9f79cfb7: Preparing 
1a8c83dd: Preparing 
acb8cf69: Preparing 
latest: digest: sha256:99738b124cc812bed9e6ba33b25bb4670121c82446914c24f5de34f463514517 size: 1791


In [ ]:
service_name = f"utec-mlops-evidentlyai-{USERNAME}"
app_runner_role = "arn:aws:iam::654654589924:role/service-role/AppRunnerECRAccessRole"

In [13]:
conf = {
    "ServiceName": service_name,
    "SourceConfiguration": {
        "AuthenticationConfiguration": {
            "AccessRoleArn": app_runner_role
        },
        "ImageRepository": {
            "ImageIdentifier": full_tag_name,
            "ImageConfiguration": {
                "Port": "8000"
            },
            "ImageRepositoryType": "ECR"
        }
    },
    "InstanceConfiguration": {
        "Cpu": "2 vCPU",
        "Memory": "4 GB"
    }
}

In [14]:
with open('app_runner_config.json', 'w') as f:
    json.dump(conf, f)

In [15]:
!aws apprunner create-service --cli-input-json file://app_runner_config.json

{
    "Service": {
        "ServiceName": "utec-mlops-evidentlyai-server",
        "ServiceId": "eeb719f6728a4693adadfe0bb21e53c3",
        "ServiceArn": "arn:aws:apprunner:us-east-1:654654589924:service/utec-mlops-evidentlyai-server/eeb719f6728a4693adadfe0bb21e53c3",
        "ServiceUrl": "nahpmgpxmm.us-east-1.awsapprunner.com",
        "CreatedAt": "2025-06-23T04:23:29.723000+00:00",
        "UpdatedAt": "2025-06-23T04:23:29.723000+00:00",
        "Status": "OPERATION_IN_PROGRESS",
        "SourceConfiguration": {
            "ImageRepository": {
                "ImageIdentifier": "654654589924.dkr.ecr.us-east-1.amazonaws.com/utec-mlops/evidentlyai/server:latest",
                "ImageConfiguration": {
                    "Port": "8000"
                },
                "ImageRepositoryType": "ECR"
            },
            "AutoDeploymentsEnabled": false,
            "AuthenticationConfiguration": {
                "AccessRoleArn": "arn:aws:iam::654654589924:role/service-role/App